In [8]:
import numpy as np
import plotly.graph_objects as go
from policyengine_us import Simulation
from policyengine_core.reforms import Reform
from policyengine_core.charts import *

In [9]:
# Define the reform
reform = Reform.from_dict({
  "gov.contrib.congress.delauro.american_family_act.baby_bonus": {
    "2024-01-01.2100-12-31": 2400
  },
  "gov.irs.credits.ctc.amount.arpa[0].amount": {
    "2023-01-01.2028-12-31": 3600
  },
  "gov.irs.credits.ctc.amount.arpa[1].amount": {
    "2023-01-01.2028-12-31": 3000
  },
  "gov.irs.credits.ctc.phase_out.arpa.in_effect": {
    "2023-01-01.2028-12-31": True
  },
  "gov.irs.credits.ctc.refundable.fully_refundable": {
    "2023-01-01.2028-12-31": True
  },
  "gov.irs.credits.ctc.amount.base[1].threshold": {
    "2024-01-01.2100-12-31": 18
  }
}, country_id="us")

In [10]:
YEAR = "2025"
MAX_INCOME = 250000

In [17]:
def create_situation(marital_status, child_age):
    situation = {
        "people": {
            "adult": {
                "age": {YEAR: 40},
            },
            "child": {
                "age": {YEAR: child_age},
            }
        },
        "families": {"family": {"members": ["adult", "child"]}},
        "marital_units": {"marital_unit": {"members": ["adult"]}},
        "tax_units": {"tax_unit": {"members": ["adult", "child"]}},
        "households": {
            "household": {"members": ["adult", "child"], "state_name": {YEAR: "TX"}}
        },
        "axes": [[
            {
                "name": "employment_income",
                "min": 0,
                "max": MAX_INCOME,
                "count": 201,
                "period": YEAR,
            }
        ]]
    }
    
    if marital_status == "married":
        situation["people"]["spouse"] = {"age": {YEAR: 40}}
        for unit in ["families", "marital_units", "tax_units", "households"]:
            situation[unit][list(situation[unit].keys())[0]]["members"].append("spouse")
        
    return situation


In [18]:
def calculate_income(situation, reform=None):
    simulation = Simulation(situation=situation, reform=reform)
    return simulation.calculate("household_net_income", YEAR)

In [21]:
def create_ctc_reform_comparison_graph():
    colors = {
        "married_0": "#0066cc",   # Dark blue
        "married_5": "#4d94ff",   # Medium blue
        "married_16": "#99c2ff",  # Light blue
        "married_17": "#cce0ff",  # Very light blue
        "single_0": "#333333",    # Dark grey
        "single_5": "#666666",    # Medium grey
        "single_16": "#999999",   # Light grey
        "single_17": "#cccccc",   # Very light grey
    }

    x = np.linspace(0, MAX_INCOME, 201)
    fig = go.Figure()

    for marital_status in ["married", "single"]:
        for child_age in [0, 5, 17]:
            baseline = calculate_income(create_situation(marital_status, child_age))
            reform_result = calculate_income(create_situation(marital_status, child_age), reform)
            
            if child_age == 0:
                child_label = "Newborn"
            elif child_age == 5:
                child_label = "Child Age 1-5"
            else:
                child_label = "Child Age 17"
            
            label = f"{marital_status.capitalize()}, {child_label}"
            color = colors[f"{marital_status}_{child_age}"]
            
            line_style = 'solid' if marital_status == "married" else 'dot'
            
            fig.add_trace(go.Scatter(
                x=x, 
                y=reform_result - baseline, 
                mode='lines', 
                name=label, 
                line=dict(color=color, dash=line_style)
            ))

    fig.update_layout(
        title='Impact of the Harris CTC Reform by Marital Status and Child Age',
        xaxis_title="Earnings",
        yaxis_title="Net Impact",
        xaxis=dict(tickformat='$,.0f', range=[0, MAX_INCOME]),
        yaxis=dict(tickformat='$,.0f'),
        legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=1.01
        ),
        height=600,
        width=800,
    )

    return fig

In [22]:
# Create and display the chart
fig = create_ctc_reform_comparison_graph()
fig = format_fig(fig)
fig.show()